In [2]:
# Installing Necessary Packages
#Pandas, Azure-storage-blob, dotenv
!python -m pip install pandas

In [3]:
!python -m pip install azure-storage-blob

In [4]:
!python -m pip install python-dotenv

In [5]:
# Importing Necessary Libraries
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os


In [6]:
# Data Extraction

try:
    data = pd.read_csv(r'zipco_transaction.csv')
    print('Data Extracted Successfully')
except Exception as e:
    print(f"An Error occurred: {e}")

Data Extracted Successfully


In [7]:
data.head()

,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,NaN,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\nEast Brandist...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 03:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 04:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

In [9]:
# Data Cleaning and Transformation
# Remove Duplicate Enteries
data.drop_duplicates(inplace=True)

In [10]:
# Handling Missing Values (Filling Missing Numeric Values with the mean or median)
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_columns:
    data.fillna({col: data[col].mean()}, inplace=True)

In [11]:
# Handling Missing Values (Fill missing string or object values with unknown)
string_columns = data.select_dtypes(include=['object']).columns
for col in string_columns:
    data.fillna({col: 'unknown'}, inplace=True)

In [34]:
data['Date'] = pd.to_datetime(data['Date'])

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   ProductName             1000 non-null   object 
 2   Quantity                1000 non-null   int64  
 3   UnitPrice               1000 non-null   float64
 4   StoreLocation           1000 non-null   object 
 5   PaymentType             1000 non-null   object 
 6   PromotionApplied        1000 non-null   bool   
 7   Weather                 1000 non-null   object 
 8   Temperature             1000 non-null   float64
 9   StaffPerformanceRating  1000 non-null   object 
 10  CustomerFeedback        1000 non-null   object 
 11  DeliveryTime_min        1000 non-null   int64  
 12  OrderType               1000 non-null   object 
 13  CustomerName            1000 non-null   object 
 14  CustomerAddress         1000 non-null   object

In [13]:
data.columns

Index(['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation',
       'PaymentType', 'PromotionApplied', 'Weather', 'Temperature',
       'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min',
       'OrderType', 'CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail', 'Staff_Name', 'Staff_Email', 'DayOfWeek',
       'TotalSales'],
      dtype='object')

In [20]:
# Create the product table
products = data[['ProductName']].drop_duplicates().reset_index(drop=True)
products.index.name = 'ProductID'
products = products.reset_index()

In [21]:
products.head(20)

,ProductID,ProductName
0,0,Vanilla Cake
1,1,Red Velvet Cake
2,2,Chocolate Cake
3,3,Carrot Cake
4,4,Pizza Pepperoni
5,5,Pizza Margherita
6,6,Pizza Beef Double decker
7,7,Cheesecake
8,8,Fruit Cake


In [22]:
# Create Customers Table
customers = data[['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber', 'CustomerEmail']].drop_duplicates().reset_index(drop=True)
customers.index.name = 'CustomerID'
customers = customers.reset_index()

In [25]:
# Staff table
staff = data[['Staff_Name', 'Staff_Email']].drop_duplicates().reset_index(drop=True)
staff.index.name = 'StaffID'
staff = staff.reset_index()

In [35]:
# Creating the transaction table
transaction = data.merge(products, on=['ProductName'], how='left') \
                  .merge(customers, on=['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber', 'CustomerEmail'], how='left') \
                  .merge(staff, on=['Staff_Name', 'Staff_Email'], how='left')

transaction.index.name = 'TransactionID'
transaction = transaction.reset_index() \
                         [['Date', 'TransactionID', 'ProductID', 'Quantity', 'UnitPrice', 'StoreLocation', 'PaymentType', 'PromotionApplied', 'Weather', 'Temperature', \
                            'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min', 'OrderType', 'CustomerID', 'StaffID', \
                            'DayOfWeek', 'TotalSales']]

In [37]:
# Save Data as CSV files
data.to_csv('clean_data.csv', index=False)
products.to_csv('products.csv', index=False)
customers.to_csv('customers.csv', index=False)
staff.to_csv('staff.csv', index=False)
transaction.to_csv('transaction.csv', index=False)